In [1]:
import numpy as np
import scipy.linalg as LA

### Import (or write): apGrad, apHess, apJacobian  (functions)

In [2]:
from AW_FD_esqueleto import apGrad, apHess, apJacobian


Imported: Finite Difference Gradient and Hessian (by AW).


In [3]:
# TEST functionality:
f = lambda x:  np.array([0.5*(x[0]+3)**2 + 0.5*(x[1]+3)**2])
#f = lambda x:  0.5*(x[0]+3)**2 + 0.5*(x[1]+3)**2
x = np.array([1,2])
print('grad:\n', apGrad(f, x) )
print('hess:\n', apHess(f, x) )
print('jaco:\n ', apJacobian(f, x) )

grad:
 [4. 5.]
hess:
 [[1. 0.]
 [0. 1.]]
jaco:
  [[4. 5.]]


### Implement: method SQP local

In [27]:
def mSQPlocal(f, h, x0, lam0, tol,itmax=100):
    """Local pure Newton with pure hessian of Lagrangian"""
    # TODO: write this function
    x=x0.copy()
    lam=lam0.copy()
    m=len(lam)
    n=len(x)
    L = lambda x: f(x)+lam.T@h(x)
    #gK=apGrad(f,x)
    #A=apJacobian(f,x)
    hk=h(x)
    assert(len(hk)==len(lam)),'lambda incompatible con el número de restricciones'
    assert(len(hk)==1),'redefine function h'
    #F(x,Lam)=gk+A.TLam; hk
    for k in range(itmax):
        gk=apGrad(f,x)
        A=apJacobian(f,x)
        hk=h(x)
        if all(abs(gk+A.T@lam)<=tol*(abs(gk)+1)) and all(abs(hk)<=tol):
            print('Happy initeration:',k)
            break
        HL=apHess(L,x)
        row1=np.hstack((HL,A.T))
        row2=np.hstack((A,np.zeros((m,m))))
        mat=np.vstack((row1,row2))
        
        rhs=np.hstack((-gk,-hk))
        delta=LA.solve(mat,rhs)
        lam=delta[n:]
        dx=delta[:n]
        x+=dx
        
        
    return x, lam



### Test: local SQP method

In [28]:
f = lambda x:  np.array([0.5*(x[0]+3)**2 + 0.5*(x[1]+3)**2])
h = lambda x:  np.array([x[0]**2 + x[1]**2 - 2])

#x0   = np.array([-2,-0])
x0   = np.array([2.0,1.0])
lam0 = np.array([0]);
tol = 1e-9;

xk, lamk = mSQPlocal( f, h, x0, lam0, tol,itmax=100 )
xk, lamk

Happy initeration: 33


(array([1.3346346 , 0.46770769]), array([-1.]))